# Coursework: Optimisation of a fantasy football team

The coursework is described in detail in the documentation provided on Moodle. This notebook contains some code for basic functions that read in the data file and define the solution/constraint checker that you must use to check your final solution.

As noted in the coursework, you don't have to use Python or DEAP to tackle this. However, the practicals have covered a lot of functionality that will be useful so you should find that the DEAP libraries provide a quick way to start and will save you some time in writing code.

## Important Information

If you use another language, then you should write out your solution to a csv file as a comma separated list of 0,1s (one value per row) indicating which players are included and then will  need to read it back to use the checker function, You report should include the screenshot of the  output from the function provided in this notebook, and *not* your own version of the function



In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

def read_file_from_directory(directory,filename):
    directory=Path(directory)
    file_path=directory/filename
    if not file_path.exists():raise FileNotFoundError(f"File not found: {file_path}")
    if not file_path.exists():raise FileNotFoundError(f"CSV file not found: {file_path}")
    return pd.read_csv(file_path).reset_index(drop=True)
    return content

# Data
The code below reads in the datafile and calculates the number of players available.  
Change the filepath to your local drive.

The file is sorted by player type. As I may check your solution **DO NOT** sort the file or alter it in any way as my code will expect to see it in this format.

Feel free to browse the file and analyse the data in any way you think might be useful

In [3]:
dir_path="."
file_name="clean-data.csv"
try:data=read_file_from_directory(dir_path,file_name)
except Exception as e:print(f"Error: {e}")
num_players=len(data.index)
budget=100
print("num possible players is %s" % (num_players))

num possible players is 523


# Helpful data
The code below extracts some useful information from the data that will be useful to you when writing your program. In particular:

- a list containing the **points** per player:  e.g. points[i] refers to the **points** associated with player *i*
- a list containing the **cost** per player: e.g. cost[i] refers to the **cost** associated with player *i*
- a list **gk** which indicates which player is a *goal-keeper*. The list is the same length as the number of players. gk[i]=0 if player *i* is not a goal-keeper; gk[i]=1 if player *i* is a goal-keeper
- a list **mid** which indicates which player is a *midfielder*. The list is the same length as the number of players. mid[i]=0 if player *i* is not a mid-fielder; mid[i]=1 if player *i* is a midfielder
- a list **defe** which indicates which player is a *defender*. The list is the same length as the number of players. defe[i]=0 if player *i* is not a defender; defe[i]=1 if player *i* is a defender
- a list **stri** which indicates which player is a *striker*. The list is the same length as the number of players. stri[i]=0 if player *i* is not a striker; stri[i]=1 if player *i* is a striker

In [4]:
points=data['Points'] 
cost=data['Cost']
gk=np.zeros(num_players)
mid=np.zeros(num_players)
defe=np.zeros(num_players)
stri=np.zeros(num_players)

for i in range(num_players):
    if data['Position'][i]=='GK':gk[i]=1
    elif data['Position'][i]=='DEF':defe[i]=1
    elif data['Position'][i]=='MID':mid[i]=1
    elif data['Position'][i]=='STR':stri[i]=1 

In [5]:
print(gk)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

# Solution and constraint checker function

You are free to represent an individiual in any way you wish during the search process. However, at the end of the evolutionary run, you *must* convert your solution to a list of length *num_players* in which each element is either 0 or 1. An element *i* should be set to 0 if player *i* is not included in the team, and to 1 if player *is* **is** included in the team.

You *must* call this function with your best solution and include a screen shot of the output in your report.

In [6]:
def check_constraints(individual):
    broken_constraints=0
    totalpoints=np.sum(np.multiply(points,individual))
    c1=np.sum(individual)
    if  c1 !=11:
        broken_constraints+=1
        print("Broken Constraint: Total number of players is %s" %(c1))
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 >100:
        broken_constraints+=1
        print("Broken Constraint: Cost is %s" %(c2))
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        print("Broken Constraint: Number of GOALIES is %s " %(c3))
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        print("Broken Constraint: Number of DEFENDERS is %s " %(c4))
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        print("Broken Constraint: Number of MID is %s " %(c5))
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        print("Broken Constraint: Number of STRIKERS is %s " %(c6))
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    if broken_constraints>0:print("INVALID SOLUTION")
    print(" ")
    print(" ")
    return broken_constraints, totalpoints, c2, selectedPlayers

# Example Function for Calculating Constraint Violations

An example function is provided below for checking all the constraints. Note this is not well designed!  It simply checks if a constraint is broken (yes/no) and returns the total number of broken constraints, the total cost, and the number of players. It does not take any account of the extent to which a constraint is broken (e.g. whether there are 12 players in the team, or 500 players in the team). Similarly for the budget, it just notes if it is exceeded - not by how much

To get good solutions, you will need to consider how to modify this function...

The function can be called from a custom evaluation function (example defined below called *footballEval*)

In [11]:
def solutionEvaluator(individual):
    totalpoints=np.sum(np.multiply(points,individual))
    penalty=0
    num_selected=np.sum(individual)
    if num_selected!=11:penalty+=abs(num_selected-11)*100
    total_cost=np.sum(np.multiply(cost,individual))
    if total_cost>100:penalty+=(total_cost-100)*50
    num_gk=np.sum(np.multiply(gk,individual))
    if num_gk!=1:penalty+=abs(num_gk-1)*200
    num_def=np.sum(np.multiply(defe,individual))
    if num_def<3:penalty+=(3-num_def)*150
    elif num_def>5:penalty+=(num_def-5)*150
    num_mid=np.sum(np.multiply(mid,individual))
    if num_mid<3:penalty+=(3-num_mid)*150
    elif num_mid>5:penalty+=(num_mid-5)*150
    num_str=np.sum(np.multiply(stri,individual))
    if num_str<1:penalty+=(1-num_str)*150
    elif num_str>3:penalty+=(num_str-3)*150
    return totalpoints,penalty,total_cost

# Basic EA using DEAP

This creates a basic EA (as per the early practicals) in which a solution is represented as a list of 523 bits, each with value 0 or 1 indicating the player at index *i* in the csv file is chosen or not.  It uses a simple initialisation procedure in which each bit has an equal probability of being set to 0 or 1 *(think about why this is not an efficient representation!)*


A custom evaluation function called *footballEval* is defined and registered.  The function assigns a crude penalty to solutions that break one or more constraints. It is very basic and unlikely to lead to good solutions:
- the number of broken constraints from the *SolutionEvaluator* function is multiplied by an arbitrary constant, and this value subtracted from the 'totalpoints' score
- if no constraints are broken, i.e. the solution is valid, the score will be positive (and should be maximised); if any constraint is broken, the score will be  a negative value

You will want to consider how to modify this function (in conjunction with the SolutionEvaluator) to better guide the EA towards good solutions

All parameters/choice of operators are arbitrary - consider how you might change these


In [8]:
import array
import numpy
import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,  toolbox.attr_bool, num_players)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def  footballEval(individual):
    broken_constraints, totalpoints, teamCost  = solutionEvaluator(individual)
    if broken_constraints > 0:penalty=10000* broken_constraints
    else:penalty=0
    return (totalpoints-penalty),

toolbox.register("evaluate", footballEval)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100,stats=stats, halloffame=hof, verbose=True)
    return pop,log,hof

In [9]:
pop, log, hof = main()


best = hof[0].fitness.values[0]
max = log.select("min")

for i in range(100):
        fit = max[i]
        if fit == best:
            break        
        
print("max fitness found is %s at gen %s " % (best, i))

gen	nevals	avg     	std   	min   	max   
0  	300   	-44676.8	1025.1	-47035	-41830
1  	187   	-43794.1	867.606	-46298	-41513
2  	183   	-43174.3	738.798	-45028	-41252
3  	179   	-42650.9	681.153	-45099	-40589
4  	169   	-42119.8	604.496	-43645	-40036
5  	192   	-41708.9	578.111	-44313	-40036
6  	195   	-41302.3	596.947	-43100	-39227
7  	164   	-40899.1	581.873	-42911	-38955
8  	193   	-40498.3	518.309	-42129	-38951
9  	182   	-40157.4	522.911	-41972	-38951
10 	175   	-39832.8	523.156	-41780	-38619
11 	196   	-39497.4	488.637	-41257	-38211
12 	166   	-39238  	422.125	-40551	-38211
13 	189   	-39048.1	483.165	-41045	-37654
14 	175   	-38776.6	528.153	-40846	-37193
15 	193   	-38498  	524.364	-40369	-37193
16 	164   	-38155.9	485.061	-39968	-36944
17 	196   	-37868.1	451.744	-39848	-36746
18 	163   	-37616.9	485.135	-39796	-36699
19 	182   	-37367.7	439.068	-39522	-36353
20 	193   	-37211.2	460.219	-39055	-36353
21 	176   	-37014.2	443.533	-38511	-36353
22 	196   	-36870.5	442.33 	-38659	-

# CHECKER

Call the code below with your best solution and include a screen shot of the output in your report

**If your code uses a representation that is different from the one in this notebook (i.e. a bitstring of length 512 where a 0/1 indicates if the player is selected for the team) then see the instruction in cell 28 below**


# call the checker to indicate which constraints are broken
broken_constraints, totalpoints, c2, selectedPlayers = check_constraints(hof[0])


print("Final Cost %s " %(c2))
print("Final Points %s" %(totalpoints))
print("Number of broken constraints %s" %(broken_constraints))
print("selected players are %s" %(selectedPlayers))
    
  

# Running the checker with a different representation

If you use a different representation, please use the code below to call the checker.  You need to supply a list with the indices of the players that you have chosen using your preferred representation. This list should contain 11 numbers (line 2 below)

In [10]:
mySelectedPlayers = [1,3,5,7,9,11,13,15,100,200,300]

individual = [0]*523

for player in mySelectedPlayers:individual[player]=1
broken_constraints, totalpoints, c2, selectedPlayers = check_constraints(individual)


print("Final Cost %s " %(c2))
print("Final Points %s" %(totalpoints))
print("Number of broken constraints %s" %(broken_constraints))
print("selected players are %s" %(selectedPlayers))   

Broken Constraint: Number of GOALIES is 0.0 
Broken Constraint: Number of DEFENDERS is 9.0 
Broken Constraint: Number of MID is 2.0 
Broken Constraint: Number of STRIKERS is 0.0 
INVALID SOLUTION
 
 
Final Cost 97.2 
Final Points 1548
Number of broken constraints 4
selected players are [1, 3, 5, 7, 9, 11, 13, 15, 100, 200, 300]
